In [1]:
import sys
sys.path.append('../codes')

# 15. 콘텍스트 관리자와 else 블록

다른 언어에서 흔히 볼 수 없기 때문에 파이썬에서도 잘 사용되지 않는..(?) 기능들을 살펴봅니다.

- `with`문과 콘텍스트 관리자
- `for`, `while`, `try` 문에서의 `else` 블록

## 15.0 `with`문

- `with` 문은 콘텍스트 관리자 객체의 제어를 받아서 임시로 콘텍스트를 생성하고 신뢰성 있게 제어합니다.
- 에러를 예방하고 반복되는 코드를 줄여주며, API를 안전하고 편리하게 사용할 수 있도록 만듭니다. 

## 15.1 이것 다음에 저것 : if 문 이외에서의 else 블록

`else` 절은 `if` 문뿐만 아니라 `for`, `while`, `try` 문에서도 사용할 수 있습니다. 

__`for`__
- `for`루프가 완전히 실행되면 `else` 블록이 실행됩니다.

In [2]:
my_lst = [1,1,1,1,1,1]
for item in my_lst:
    if item == 2:
        break
else:
    raise ValueError('No 2 found')

ValueError: No 2 found

__`while`__
- `while` 루프를 빠져나온 후에 `else` 블록 실행

In [3]:
while my_lst:
    my_lst.pop()
else:
    print('my_lst is empty')

my_lst is empty


__`try`__
- `try` 블록에서 예외가 발생하지 않을 때만 `else` 블록이 실행됩니다. 
- `else` 블록에서 발생한 예외는 `else` 블록 앞에 나오는 `except` 블록에서 처리되지 않습니다.

In [4]:
def after_call():
    print('after call')
try:
    dangerous_call()
    after_call()
except NameError:
    print('name error')

name error


`try` 문에서 `else`는 필요 없어 보일 수 있지만, 코드의 의도를 명확히 표현하기 위해 `else` 블록을 줍니다.

In [5]:
def after_call():
    print('after call')
try:
    dangerous_call()
except NameError:
    print('name error')
else:
    after_call()

name error


파이썬에서는 `try`/`except` 를 예외 처리뿐만 아니라 일반적인 제어 흐름을 구현하기 위해서도 많이 사용합니다. 

__EAPP__

_허락을 구하기보다 용서를 구하는 것이 더 쉽다._ 어떤 가정이 잘못되었을 때는 예외를 잡아서 처리하는 파이썬의 코딩스타일. `try`/`except` 문을 많이 사용합니다.

__LBYL__

_누울 자리를 보고 다리를 뻗으라._ 호출이나 조회를 하기 전에 전제 조건을 명시적으로 검사합니다. EAFP와 대조적이며 `if`문을 많이 사용합니다.

## 15.2 콘텍스트 관리자와 `with` 블록

반복자가 `for`문을 제어하기 위해 존재하는 것과 마찬가지로, 콘텍스트 관리자 객체는 `with` 문을 제어하기 위해 존재합니다.

`with` 문은 try/finally 패턴을 단순화하기 위해 설계되었습니다.

> try/finally 패턴 : 예외, `return`, `sys.exit()` 호출 등의 이유로 어떤 블록의 실행이 중단되더라도 이후의 일정한 코드를 반드시 실행할 수 있도록 보장 
>
>일반적으로 finally 절 안에 있는 코드는 중요한 리소스를 해제하거나 임시로 변경된 상태를 복원하기 위해 사용

콘텍스트 관리자 프로토콜은 `__enter__()`와 `__exit__()` 메서드로 구성. 

`with`문이 시작될 때 콘텍스트 관리자 객체의 `__enter__()` 메서드가 호출. 이 메서드는 `with` 블록의 끝에서 finally 절의 역할을 수행

In [6]:
# ex 15.1 콘텍스트 관리자로서 파일 객체의 사용 예
with open('../codes/zen.txt') as fp:
    src = fp.read(60)

파일의 `__enter__()` 메서드가 `self`를 반환하므로 `fp`는 열린 파일에 바인딩 됩니다.

In [7]:
len(src)

60

In [8]:
fp

<_io.TextIOWrapper name='../codes/zen.txt' mode='r' encoding='UTF-8'>

In [9]:
fp.closed, fp.encoding

(True, 'UTF-8')

`fp` 변수는 여전히 살아 있습니다.

In [10]:
fp.read(60)

ValueError: I/O operation on closed file.

`with` 블록이 끝날 때 `TextIOWrapper.__exit__()` 메서드가 호출되어 파일을 닫으므로 fp를 이용해서 파일 입출력을 할 수 없습니다.

- `open()` 함수가 `TextIOWrapper` 객체를 반환하고, 이 객체의 `__enter__()` 메서드는 `self`를 반환합니다.
- `with` 문을 빠져나온 후에는 `__enter__()` 메서드가 반환한 객체가 아니라 콘텍스트 관리자 객체의 `__exit__()` 메서드가 호출
- `with`문의 `as`절은 선택적 `open()` 의 경우, 파일에 대한 참조가 필요하지만, 사용자에게 반환할 적절한 객체가 없어 `None`을 반환할 수도 있음

In [11]:
# ex 15.2 LookingGlass 콘텍스트 관리자 클래스의 시험 주행
from mirror import LookingGlass
with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


`LookingGlass` 객체가 콘텍스트 관리자입니다. 파이썬은 콘텍스트 관리자의 `__enter__()` 메서드를 호출해서 반환된 값을 `what`에 바인딩합니다.

In [12]:
what

'JABBERWOCKY'

`with` 블록이 끝났으니 `__enter__()`가 반환해서 `what`에 저장한 문자열 `JABBERWOCKY`를 제대로 출력할 수 있습니다.

In [13]:
print('Back to normal.')

Back to normal.


In [14]:
!cat ../codes/mirror.py

class LookingGlass:
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'

    def reverse_write(self, text):
        self.original_write(text[::-1])

    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True


파이썬 인터프리터는 `__enter__()` 메서드를 호출할 때 `self` 이외의 인수는 전달하지 않습니다. 

`__exit__()` 메서드를 호출할 때는 다음 세 인수를 전달합니다. 

- `exc_type` : `ZeroDivisionError` 등의 예외 클래스
- `exc_value` : 예외 객체, 예외 메시지 등 `exception()` 생성자에 전달된 인수는 `exc_value.args` 속성을 이용해서 볼 수 있습니다. 
- `traceback` : traceback 객체

In [15]:
# ex 15.4 with 블록 없이 LookingGlass 사용하기
from mirror import LookingGlass
manager = LookingGlass()
manager

In [16]:
monster = manager.__enter__()
monster == 'JABBERWOCKY'

True

In [17]:
monster

'JABBERWOCKY'

In [18]:
manager

In [19]:
manager.__exit__(None, None, None)

In [20]:
monster

'JABBERWOCKY'

콘텍스트 관리자는 상당히 독특한 기능으로서, 느리지만 이 기능을 활용하기 위해 노력하고 있습니다.
- `sqlite3` 모듈의 트랜잭션 관리
- `threading` 코드에서 `lock`, `condition`, `semaphore` 보관
- `Decimal` 객체의 산술 연산 환경 설정
- 객체의 테스트를 위한 임시 패치 적용

### 15.3 `contextlib` 유틸리티

`contextlib` 모듈에는 `redirect_stdout` 외에도 다양하게 응용할 수 있는 클래스와 함수가 있습니다.

- `closing()` :  `close()` 메서드는 제공하지만 `__enter__()`/`__exit__()` 프로토콜을 구현하지 않는 객체로부터 콘텍스트 관리자를 생성하는 함수


- `suppress` : 지정한 예외를 임시로 무시하는 콘텍스트 관리자


- `@contextmanager` : 클래스를 생성하고 프로토콜을 구현하는 대신, 간단한 제너레이터 함수로부터 콘텍스트 관리자를 생성할 수 있게 해주는 데커레이터


- `ContextDecorator` : 콘텍스트 관리자를 함수 데커레이터로도 사용할 수 있게 해주는 기반 클래스


- `ExitStack` : 여러 콘텍스트 관리자를 입력할 수 있게 해주는 콘텍스트 관리자. `with` 블록이 끝나면 `ExitStack`은 누적된 콘텍스트 관리자를 관리자들의 `__exit__()` 메서드를 LIFO순서로 호출합니다.

### 15.4 `@contextmanager` 사용하기 

`@contextmanager` 데커레이터는 콘텍스트 관리자를 생성할 때 작성하는 코드를 줄여줍니다.

`__enter__()`와 `__exit__()` 메서드를 가진 클래스 전체를 구현하는 대신 `__enter__()` 메서드가 반환할 것을 생성하는 `yield` 문 하나를 가진 제너레이터만 구현하면 됩니다. 

In [21]:
!cat ../codes/mirror_gen.py

import contextlib

@contextlib.contextmanager #1
def looking_glass():
    import sys
    original_write = sys.stdout.write #2
    
    def reverse_write(text): #3
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write #4
    yield 'JABBERWOCKY' #5
    sys.stdout.write = original_write #6


1. `@contextmanager` 데커레이터를 적용합니다.


2. 원래의 `sys.stdout.write()` 메서드를 보관합니다.


3. `reverse_write()` 함수를 정의. `original_write()`는 클로저를 통해 접근할 수 있습니다.


4. `sys.stdout.write()`를 `reverse_write()`로 교체


5. `with` 문의 `as` 절에 있는 타깃 변수에 바인딩될 값을 생성. `with` 문의 본체가 실행되는 동안 이 함수는 여기서 실행을 일시 중단


6. 제어 흐름이 `with` 블록을 빠져나오면 `yield` 문 이후의 코드가 실행. 여기서는 원래의 `sys.stdout.write()` 메서드를 복원

여기서 `yield`는 함수를 두 부분으로 나누기 위해 사용됩니다.

`yield` 이전은 `with` 블록 앞에서 인터프리터가 `__enter__()`를 호출할 때 실행되고, 이후는 블록의 마지막에서 `__exit__()`가 호출될 때 실행됩니다.

In [22]:
## ex 15-6
from mirror_gen import looking_glass
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [23]:
what

'JABBERWOCKY'

이 클래스의 `__enter__()` 메서드는 다음과 같은 단계를 실행합니다.
1. 제너레이터 함수를 호출해서 제너레이터 객체를 보관
2. `next(gen)`을 호출해서 `yield` 키워드 앞까지 실행
3. `next(gen)`이 생성한 값을 반환해서, 이 값이 `as` 절의 타깃 변수에 바인딩 되게 합니다.

이 클래스의 `__exit__()` 메서드는 다음과 같은 단계를 실행합니다.
1. `exc_type`에 예외가 전달되었는지 확인. 만일 그렇다면 제너레이터 함수 본체 안에 있는 `yiled` 행에서 `gen.throw(exception)`를 실행해서 예외를 발생 시킨 것
2. 그렇지 않다면 `next(gen)`을 호출해서 제너레이터 함수 본체 안의 `yield` 다음의 코드를 계속 실행

#### 예외 처리를 구현한 제너레이터 기반 콘텍스트 관리자

In [24]:
! cat ../codes/mirror_gen_exc.py

import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
    
    sys.stdout.write = reverse_write
    msg = '' #1
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError: #2
        msg = 'Please DO NOT divice by zero !'
    finally:
        sys.stdout.write = original_write #3
        if msg:
            print(msg) #4


1. 에러 메시지에 대한 변수


2. 에러 메시지를 설정해서 `ZeroDivisionError` 처리


3. 멍키 패칭한 `sys.stdout.write()` 를 원래대로 복원


4. 에러 메시지가 설정되어 있으면 출력

`__exit__()` 메서드는 예외 처리를 완료했음을 인터프리터에 알려주기 위해 `True`를 반환합니다.

`True`가 반환되면 인터프리터는 예외를 전파하지 않고 억제합니다. 한편 `__exit__()`가 명시적으로 값을 반환하지 않으면 인터프리터가 `None`을 받으므로 예외를 전파합니다.

데커레이터가 제공하는 `__exit__()` 메서드는 제너레이터에 전달된 예외가 모두 처리되었으므로 억제해야 한다고 생각합니다.

```
## ex 15-8
import csv

with inplace(csv_fname, 'r', newline='') as (infh, outfh):
    reader = csv.reader(infh)
    writer = csv.writer(outfh)
    
    for row in reader:
        row += ['new', 'columns']
        writer.writerow(row)
        
```

`inplace()` 함수가 콘텍스트 관리자며, 동일한 파일에 대해 두 개의 핸들(`infh`, `outfh`) 을 반환함으로써 파일을 동시에 읽고 쓸 수 있게 해줍니다. 


이 코드는 마르틴 피터스가 작성한 __파일 덮어쓰기 콘텍스트 관리자__ 입니다. http://bit.ly/1MM96aR


이 코드를 분석하고 싶으면 `yield` 키워드를 찾아서, 앞의 코드는 콘텍스트를 설정하고, 백업 파일을 생성하고, 파일을 연 후에 `__enter__()`가 호출되면 반환할 읽기/쓰기 용 파일 핸들에 대한 참조를 생성합니다. 


`__exit__()` 가 수행하는 `yield` 뒤의 코드는 파일을 닫고, 문제가 생긴 경우에는 백업 파일로 복구하는 작업을 수행합니다. 

### 15.5 요약

1. `for`, `while`, `try` 문에서 사용하는 `else` 블록


2. 콘텍스트 관리자와 `with` 문의 의미. 
> `with`문이 단지 리소스 관리를 위한 것이 아니라, 공통적인 준비와 마무리 작업 또는 다른 프로시저의 전과 후에 수행해야 할 연산을 인수 분해하는 도구 .... ? by 레이몬드 헤팅거. PyCon US 2013


3. `contextlib` 표준 라이브러리 모듈에 있는 함수들. 그 중 `@contextmanager`


